In [1]:
import pandas as pd

def pdb_to_df(file_name):
    df = pd.DataFrame(columns = ['Protein_Num','Protein_Name','Cite_Name','x_coord','y_coord','z_coord'])
    with open(file_name, 'r') as file:
        index = 0
        for line in file.readlines():
            line = line.split(' ')
            if line[0] == 'ATOM':
                info = []
                for i in line:
                    if i != '':
                        info.append(i)
                df.loc[index, 'Protein_Num'] = int(info[4])
                df.loc[index, 'Protein_Name'] = info[3]
                df.loc[index, 'Cite_Name'] = info[2]
                df.loc[index, 'x_coord'] = float(info[5])
                df.loc[index, 'y_coord'] = float(info[6])
                df.loc[index, 'z_coord'] = float(info[7])
            index += 1
        df = df.dropna()
        df = df.drop(index = df[(df.Cite_Name == 'COM')].index.tolist())
        df = df.reset_index(drop = True)
    return df

In [2]:
def dis_cal(x, y):
    return ((x[0]-y[0])**2+(x[1]-y[1])**2+(x[2]-y[2])**2)**0.5

In [3]:
def dis_df_gen(df, info):
    dis_df = pd.DataFrame(columns = ['Protein_Num_1','Protein_Name_1','Cite_Name_1','Protein_Num_2','Protein_Name_2','Cite_Name_2','sigma','dis'])
    index = 0
    count = 1
    for i in range(len(info)):
        df_temp_1 = df[df['Protein_Name'].isin([info.iloc[i, 0]])]
        df_1 = df_temp_1[df_temp_1['Cite_Name'].isin([info.iloc[i, 1]])]
        df_temp_2 = df[df['Protein_Name'].isin([info.iloc[i, 2]])]
        df_2 = df_temp_2[df_temp_2['Cite_Name'].isin([info.iloc[i, 3]])]
        df_1 = df_1.reset_index(drop = True)
        df_2 = df_2.reset_index(drop = True)
        print('Calculating distance for reaction #', count,'...')
        count += 1
        for j in range(len(df_1)):
            for k in range(len(df_2)):
                dis_df.loc[index, 'Protein_Num_1'] = df_1.loc[j, 'Protein_Num']
                dis_df.loc[index, 'Protein_Name_1'] = df_1.loc[j, 'Protein_Name']
                dis_df.loc[index, 'Cite_Name_1'] = df_1.loc[j, 'Cite_Name']
                dis_df.loc[index, 'Protein_Num_2'] = df_2.loc[k, 'Protein_Num']
                dis_df.loc[index, 'Protein_Name_2'] = df_2.loc[k, 'Protein_Name']
                dis_df.loc[index, 'Cite_Name_2'] = df_2.loc[k, 'Cite_Name']
                dis_df.loc[index, 'sigma'] = info.loc[i, 'sigma']
                x = [df_1.loc[j, 'x_coord'], df_1.loc[j, 'y_coord'], df_1.loc[j, 'z_coord']]
                y = [df_2.loc[k, 'x_coord'], df_2.loc[k, 'y_coord'], df_2.loc[k, 'z_coord']]
                dis_df.loc[index, 'dis'] = dis_cal(x, y)
                index += 1

    return dis_df

In [4]:
def bind_df_gen(dis_df, buffer_ratio):
    bind_df = pd.DataFrame(columns = ['Protein_Num_1','Protein_Name_1','Cite_Name_1','Protein_Num_2','Protein_Name_2','Cite_Name_2','sigma','dis'])
    index = 0
    for i in range(len(dis_df)):
        if dis_df.loc[i, 'dis'] >= dis_df.loc[i, 'sigma']*(1-buffer_ratio):
            if dis_df.loc[i, 'dis'] <= dis_df.loc[i, 'sigma']*(1+buffer_ratio):
                bind_df.loc[index] = dis_df.loc[i]
                index += 1
    return bind_df

In [5]:
def find_bond(bind_df):
    bond_lst = []
    for i in range(len(bind_df)):
        bond_lst.append([int(bind_df.loc[i, 'Protein_Num_1']), int(bind_df.loc[i, 'Protein_Num_2'])])
    for i in bond_lst:
        i.sort()
    bond_lst_ = []
    for i in bond_lst:
        if i not in bond_lst_:
            bond_lst_.append(i)
    return bond_lst_

In [6]:
def find_complex(pdb_df, bond_lst):
    complex_lst = []
    for i in range(1, 1+pdb_df['Protein_Num'].max()):
        complex_temp = [i]
        j = 0
        while j < len(bond_lst):
            if bond_lst[j][0] in complex_temp and bond_lst[j][1] not in complex_temp:
                complex_temp.append(bond_lst[j][1])
                j = 0
            elif bond_lst[j][1] in complex_temp and bond_lst[j][0] not in complex_temp:
                complex_temp.append(bond_lst[j][0])
                j = 0
            else:
                j += 1
        complex_lst.append(complex_temp)
    for i in complex_lst:
        i.sort()
    complex_lst_ = []
    for i in complex_lst:
        if i not in complex_lst_:
            complex_lst_.append(i)       
    return complex_lst_

In [7]:
def complex_df_gen(pdb_df, complex_lst):
    name_lst = list(pdb_df['Protein_Name'])
    name_lst_ = []
    for i in name_lst:
        if i not in name_lst_:
            name_lst_.append(i)
    column_lst = []
    for i in name_lst_:
        column_lst.append(i)
    column_lst.append('Protein_Num')
    complex_df = pd.DataFrame(columns = column_lst)
    index = 0
    for i in complex_lst:
        complex_df.loc[index] = 0
        complex_df.loc[index,'Protein_Num'] = str(i)
        for j in i:
            for indexs in pdb_df.index: 
                for k in range(len(pdb_df.loc[indexs].values)): 
                    if(pdb_df.loc[indexs].values[k] == j): 
                        col = pdb_df.loc[indexs,'Protein_Name']
                        complex_df.loc[index,col] += 1
                        break
                else:
                    continue
                break
        index += 1 
    return complex_df

In [8]:
def find_complex_df(complex_df, num_lst, pdb_df):
    protein_name = []
    for i in range(len(pdb_df)):
        if pdb_df.loc[i, 'Protein_Name'] not in protein_name:
            protein_name.append(pdb_df.loc[i, 'Protein_Name'])
    complex_df['Num_List'] = ''
    for i in range(complex_df.shape[0]):
        lst = []
        for j in range(complex_df.shape[1]-2):
            lst.append(complex_df.iloc[i, j])
        complex_df.loc[i, 'Num_List'] = str(lst)
    num_lst_str = str(num_lst)
    protein_remain = []
    for i in range(complex_df.shape[0]):
        if complex_df.loc[i, 'Num_List'] == num_lst_str:
            string = complex_df.loc[i, 'Protein_Num']
            string = string.strip('[').strip(']').split(',')
            for i in string:
                protein_remain.append(int(i))           
    return protein_remain

In [9]:
def new_pdb(file_name, protein_remain):
    with open(file_name, 'r') as file:
        write_lst = []
        for line in file.readlines():
            line_ = line.split(' ')
            if line_[0] == 'TITLE':
                write_lst.append(line)
            elif line_[0] == 'CRYST1':
                write_lst.append(line)
            elif line_[0] == 'ATOM':
                info = []
                for i in line_:
                    i.strip('\n')
                    if i != '':
                        info.append(i)
                info[9] = info[9].strip('\n')
                if int(info[4]) in protein_remain:
                    write_lst.append(line)
    with open('output_file.pdb','w') as file_:
        file_.seek(0)
        file_.truncate()
        for i in write_lst:
            file_.writelines(i)
    return 0

In [10]:
def binding_info_df(inp_name):
    status = False
    index = -1
    binding_info = pd.DataFrame(columns = ['Protein_Name_1','Cite_Name_1','Protein_Name_2','Cite_Name_2','sigma'])
    with open(inp_name, 'r') as file:
        for line in file.readlines():
            if line == 'end reactions\n':
                status = False
                break
            if line == 'start reactions\n':
                status = True
            if status:
                if '<->' in line:
                    index += 1
                    line1 = line.split('+')
                    element1 = line1[0].strip(' ')
                    line2 = line1[1].split('<->')
                    element2 = line2[0].strip(' ')
                    element1_ = element1.strip(')').split('(')
                    element2_ = element2.strip(')').split('(')
                    binding_info.loc[index, 'Protein_Name_1'] = element1_[0][0:3]
                    binding_info.loc[index, 'Cite_Name_1'] = element1_[1][0:3]
                    binding_info.loc[index, 'Protein_Name_2'] = element2_[0][0:3]
                    binding_info.loc[index, 'Cite_Name_2'] = element2_[1][0:3]
                if 'sigma' in line:
                    binding_info.loc[index, 'sigma'] = float(line.split(' = ')[-1].strip('\n'))
    return binding_info

In [11]:
def locate_position_PDB(pdb_name, num_lst, inp_name = 'parms.inp', buffer_ratio = 0.01):
    print('Reading files...')
    pdb_df = pdb_to_df(pdb_name)
    print('Reading files complete!')
    print('Extracting binding information...')
    binding_info = binding_info_df(inp_name)
    print('Extracting complete!')
    print('Calculating distance...')
    dis_df = dis_df_gen(pdb_df, binding_info)
    print('Calculation complete!')
    print('Finding bonds...')
    bind_df = bind_df_gen(dis_df, buffer_ratio)
    bond_lst = find_bond(bind_df)
    print('Finding bonds complete!')
    print('Finding complexes...')
    complex_lst = find_complex(pdb_df, bond_lst)
    complex_df = complex_df_gen(pdb_df, complex_lst)
    print('Finding complexes complete!')
    print('Writing new PDB files...')
    protein_remain = find_complex_df(complex_df, num_lst, pdb_df)
    new_pdb(pdb_name, protein_remain)
    print('PDB writing complete!(named as output_file.pdb)')

In [12]:
locate_position_PDB(pdb_name = '9999999.pdb',num_lst = [8], inp_name = 'parms.inp', buffer_ratio = 0.01)

Reading files...
Reading files complete!
Extracting binding information...
Extracting complete!
Calculating distance...
Calculating distance for reaction # 1 ...
Calculating distance for reaction # 2 ...
Calculating distance for reaction # 3 ...
Calculating distance for reaction # 4 ...
Calculating distance for reaction # 5 ...
Calculating distance for reaction # 6 ...
Calculation complete!
Finding bonds...
Finding bonds complete!
Finding complexes...
Finding complexes complete!
Writing new PDB files...
PDB writing complete!(named as output_file.pdb)
